In [1]:
import tensorflow as tf
import time
import pandas as pd
import os
import numpy as np
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Dropout, Flatten, Dense, \
    GlobalAveragePooling2D
from tensorflow.keras import Model
from matplotlib import pyplot as plt
from collections import Counter
from imblearn.over_sampling import BorderlineSMOTE
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [4]:
all_df=pd.read_csv("E:\data\multiclass\multiclass\data9.1.csv")
tr_Y = all_df['marker']
tr_X = all_df.drop('marker', axis=1)
sm = BorderlineSMOTE(random_state=42,kind="borderline-1")
tr_X, tr_Y = sm.fit_resample(tr_X, tr_Y)
all_df=pd.concat([tr_X,tr_Y],axis=1)
ndarray = all_df.values
#把特征和标签分开用于归一化
label=ndarray[:,-1].reshape(ndarray.shape[0],1)
feature=ndarray[:,:-1]
from sklearn import preprocessing
#数据归一化
minmax_scale=preprocessing.MinMaxScaler(feature_range=(0,1))
scalefeature=minmax_scale.fit_transform(feature)
#先将数据集进行拼接，要不然我们只针对样本进行采样的话，会找不到对应的标签的
data = np.hstack((scalefeature,label))
#使用随机采样方式划分数据集
train_set,test_set = train_test_split(data,test_size = 0.2,random_state = 42)
n=len(train_set)
m=len(test_set)
#把训练和测试的特征和标签分开
tr_feature=train_set[:,:-1]
tr_label=train_set[:,-1].reshape(n,1)
te_feature=test_set[:,:-1]
te_label=test_set[:,-1].reshape(m,1)

b=np.zeros(tr_feature.shape[0]*16).reshape(tr_feature.shape[0],16)
a=np.hstack((tr_feature,b)).reshape(tr_feature.shape[0],12,12)
# 给数据增加一个维度，使数据和网络结构匹配
x_train = a.reshape(a.shape[0], 12, 12, 1)
#x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
print("x_train.shape", x_train.shape)
tr_label=tr_label.astype(np.uint8)
b=np.zeros(te_feature.shape[0]*16).reshape(te_feature.shape[0],16)
a=np.hstack((te_feature,b)).reshape(te_feature.shape[0],12,12)
# 给数据增加一个维度，使数据和网络结构匹配
x_test = a.reshape(a.shape[0], 12, 12, 1)
#x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
print("x_train.shape", x_test.shape)
te_label=te_label[:,-1].astype(np.uint8)

class ConvBNRelu(Model):
    def __init__(self, ch, kernelsz=3, strides=1, padding='same'):
        super(ConvBNRelu, self).__init__()
        self.model = tf.keras.models.Sequential([
            Conv2D(ch, kernelsz, strides=strides, padding=padding),
            
            BatchNormalization(),
            Activation('relu')
        ])

    def call(self, x): 
        #在training=False时，BN通过整个训练集计算均值、方差去做批归一化，training=True时，通过当前batch的均值、方差去做批归一化。推理时 training=False效果好
        x = self.model(x, training=False)
        return x
    
#通过设定少于输入特征图深度的1*1卷积核个数减少了特征图深度，起到了降维的作用
class InceptionBlk(Model):
    def __init__(self, ch, strides=1):
        super(InceptionBlk, self).__init__()
        self.ch = ch
        self.strides = strides
        #第一分支：16个1*1卷积核，步长为1，全0填充
        self.c1 = ConvBNRelu(ch, kernelsz=1, strides=strides)
        #第二分支：16个1*1卷积核降维和16个3*3卷积核
        self.c2_1 = ConvBNRelu(ch, kernelsz=1, strides=strides)
        self.c2_2 = ConvBNRelu(ch, kernelsz=3, strides=1)
        #第三分支：16个1*1卷积核降维和16个5*5卷积核
        self.c3_1 = ConvBNRelu(ch, kernelsz=1, strides=strides)
        self.c3_2 = ConvBNRelu(ch, kernelsz=5, strides=1)
        #第四分支：3*3最大池化核和16个1*1卷积核降维
        self.p4_1 = MaxPool2D(3, strides=1, padding='same')
        self.c4_2 = ConvBNRelu(ch, kernelsz=1, strides=strides)

    def call(self, x):
        x1 = self.c1(x)
        x2_1 = self.c2_1(x)
        x2_2 = self.c2_2(x2_1)
        x3_1 = self.c3_1(x)
        x3_2 = self.c3_2(x3_1)
        x4_1 = self.p4_1(x)
        x4_2 = self.c4_2(x4_1)
        
        #卷积连接器，按深度方向堆叠构成inception结构块的输出
        x = tf.concat([x1, x2_2, x3_2, x4_2], axis=3)
        return x

class Inception10(Model):
    def __init__(self, num_blocks, num_classes, init_ch=16, **kwargs):
        super(Inception10, self).__init__(**kwargs)
        self.in_channels = init_ch
        self.out_channels = init_ch
        self.num_blocks = num_blocks
        self.init_ch = init_ch
        #网络第一层，16个3*3卷积核
        self.c1 = ConvBNRelu(init_ch)
        self.blocks = tf.keras.models.Sequential()
        #4个inception结构块顺序相连，每两个结构块组成一个block
        for block_id in range(num_blocks):
            #block_0设置的通道数为16，经过四个分支输出的深度为4*16=64
            #block_1的通道数是32，经过四个分支输出深度是4*32=128
            for layer_id in range(2):
                if layer_id == 0:
                    #每个block中的第一个inception结构块卷积步长是2，输出特征图尺寸减半
                    block = InceptionBlk(self.out_channels, strides=2)
                else:
                    #第二个inception结构块卷积步长是1
                    block = InceptionBlk(self.out_channels, strides=1)
                self.blocks.add(block)
            #加深输出特征图深度，尽可能保证特征抽取中信息的承载量一致
            #通道数加倍，故block_1通道数是block_0通道数两倍
            self.out_channels *= 2
        #平均池化
        self.p1 = GlobalAveragePooling2D()
        #5个分类的全连接
        self.f1 = Dense(num_classes, activation='softmax')

    def call(self, x):
        x = self.c1(x)
        x = self.blocks(x)
        x = self.p1(x)
        y = self.f1(x)
        return y


x_train.shape (8800, 12, 12, 1)
x_train.shape (2200, 12, 12, 1)


In [5]:
start = time.time()

model = Inception10(num_blocks=2, num_classes=5)
#编译模型
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['sparse_categorical_accuracy'])

#filepath="CNN_best_weights.{epoch:02d}-{val_sparse_categorical_accuracy:.4f}.hdf5"
# checkpoint = ModelCheckpoint(filepath, monitor='val_sparse_categorical_accuracy', verbose=1, 
#                              save_weights_only=True, save_best_only=True, mode='max')

#validation_data指定测试集，validation_freq指定测试频率（多少epoch进行一次验证）
history = model.fit(x_train, tr_label, batch_size=10, epochs=200, validation_data=(x_test, te_label))


#输出模型各层的参数状况
model.summary()

time_used = time.time()-start
print(time_used)

Train on 8800 samples, validate on 2200 samples
Epoch 1/200
8800/8800 [==============================] - 12s 1ms/sample - loss: 1.4985 - sparse_categorical_accuracy: 0.3143 - val_loss: 1.4381 - val_sparse_categorical_accuracy: 0.3241
Epoch 2/200
8800/8800 [==============================] - 10s 1ms/sample - loss: 1.3056 - sparse_categorical_accuracy: 0.4445 - val_loss: 1.2246 - val_sparse_categorical_accuracy: 0.4886
Epoch 3/200
8800/8800 [==============================] - 9s 1ms/sample - loss: 1.1343 - sparse_categorical_accuracy: 0.5305 - val_loss: 1.0683 - val_sparse_categorical_accuracy: 0.5605
Epoch 4/200
8800/8800 [==============================] - 10s 1ms/sample - loss: 1.0031 - sparse_categorical_accuracy: 0.5935 - val_loss: 0.9406 - val_sparse_categorical_accuracy: 0.6168
Epoch 5/200
8800/8800 [==============================] - 10s 1ms/sample - loss: 0.9076 - sparse_categorical_accuracy: 0.6318 - val_loss: 0.8606 - val_sparse_categorical_accuracy: 0.6436
Epoch 6/200
8800/8800 [

Epoch 45/200
8800/8800 [==============================] - 9s 1ms/sample - loss: 0.2018 - sparse_categorical_accuracy: 0.9200 - val_loss: 0.3451 - val_sparse_categorical_accuracy: 0.8845
Epoch 46/200
8800/8800 [==============================] - 9s 1ms/sample - loss: 0.2086 - sparse_categorical_accuracy: 0.9182 - val_loss: 0.3780 - val_sparse_categorical_accuracy: 0.8923
Epoch 47/200
8800/8800 [==============================] - 9s 1ms/sample - loss: 0.2007 - sparse_categorical_accuracy: 0.9202 - val_loss: 0.3237 - val_sparse_categorical_accuracy: 0.8873
Epoch 48/200
8800/8800 [==============================] - 10s 1ms/sample - loss: 0.2094 - sparse_categorical_accuracy: 0.9173 - val_loss: 0.3314 - val_sparse_categorical_accuracy: 0.8859
Epoch 49/200
8800/8800 [==============================] - 9s 1ms/sample - loss: 0.1910 - sparse_categorical_accuracy: 0.9257 - val_loss: 0.3538 - val_sparse_categorical_accuracy: 0.8759
Epoch 50/200
8800/8800 [==============================] - 9s 1ms/samp

8800/8800 [==============================] - 9s 1ms/sample - loss: 0.1496 - sparse_categorical_accuracy: 0.9417 - val_loss: 0.3000 - val_sparse_categorical_accuracy: 0.9150
Epoch 90/200
8800/8800 [==============================] - 9s 1ms/sample - loss: 0.1303 - sparse_categorical_accuracy: 0.9470 - val_loss: 0.3663 - val_sparse_categorical_accuracy: 0.8882
Epoch 91/200
8800/8800 [==============================] - 9s 1ms/sample - loss: 0.1320 - sparse_categorical_accuracy: 0.9475 - val_loss: 0.3137 - val_sparse_categorical_accuracy: 0.9159
Epoch 92/200
8800/8800 [==============================] - 9s 1ms/sample - loss: 0.1438 - sparse_categorical_accuracy: 0.9427 - val_loss: 0.3523 - val_sparse_categorical_accuracy: 0.9132
Epoch 93/200
8800/8800 [==============================] - 9s 1ms/sample - loss: 0.1139 - sparse_categorical_accuracy: 0.9556 - val_loss: 0.3763 - val_sparse_categorical_accuracy: 0.9018
Epoch 94/200
8800/8800 [==============================] - 9s 1ms/sample - loss: 0.1

Epoch 133/200
8800/8800 [==============================] - 9s 1ms/sample - loss: 0.1088 - sparse_categorical_accuracy: 0.9573 - val_loss: 0.3081 - val_sparse_categorical_accuracy: 0.9259
Epoch 134/200
8800/8800 [==============================] - 9s 1ms/sample - loss: 0.1035 - sparse_categorical_accuracy: 0.9589 - val_loss: 0.3018 - val_sparse_categorical_accuracy: 0.9318
Epoch 135/200
8800/8800 [==============================] - 9s 1ms/sample - loss: 0.1001 - sparse_categorical_accuracy: 0.9588 - val_loss: 0.2773 - val_sparse_categorical_accuracy: 0.9359
Epoch 136/200
8800/8800 [==============================] - 9s 1ms/sample - loss: 0.1072 - sparse_categorical_accuracy: 0.9570 - val_loss: 0.3164 - val_sparse_categorical_accuracy: 0.9273
Epoch 137/200
8800/8800 [==============================] - 10s 1ms/sample - loss: 0.1115 - sparse_categorical_accuracy: 0.9549 - val_loss: 0.2678 - val_sparse_categorical_accuracy: 0.9336
Epoch 138/200
8800/8800 [==============================] - 9s 1m

Epoch 177/200
8800/8800 [==============================] - 9s 1ms/sample - loss: 0.0928 - sparse_categorical_accuracy: 0.9657 - val_loss: 0.4491 - val_sparse_categorical_accuracy: 0.8832
Epoch 178/200
8800/8800 [==============================] - 9s 1ms/sample - loss: 0.0941 - sparse_categorical_accuracy: 0.9631 - val_loss: 0.3284 - val_sparse_categorical_accuracy: 0.9327
Epoch 179/200
8800/8800 [==============================] - 9s 1ms/sample - loss: 0.0874 - sparse_categorical_accuracy: 0.9644 - val_loss: 0.3051 - val_sparse_categorical_accuracy: 0.9241
Epoch 180/200
8800/8800 [==============================] - 9s 1ms/sample - loss: 0.0958 - sparse_categorical_accuracy: 0.9634 - val_loss: 0.2789 - val_sparse_categorical_accuracy: 0.9386
Epoch 181/200
8800/8800 [==============================] - 9s 1ms/sample - loss: 0.0841 - sparse_categorical_accuracy: 0.9657 - val_loss: 0.3046 - val_sparse_categorical_accuracy: 0.9327
Epoch 182/200
8800/8800 [==============================] - 9s 1ms

In [6]:
pred_y=model.predict(x_test, batch_size=100)
#返回索引
pred_yindex=np.argmax(pred_y, axis=1)

In [12]:
#1
print(sklearn.metrics.classification_report(te_label,pred_yindex))

              precision    recall  f1-score   support

           0       0.98      0.99      0.98       542
           1       0.93      0.93      0.93       554
           2       0.95      0.94      0.94       536
           3       0.91      1.00      0.95       525
           4       0.97      0.88      0.92       531

    accuracy                           0.95      2688
   macro avg       0.95      0.95      0.95      2688
weighted avg       0.95      0.95      0.95      2688



In [16]:
#2
score = model.evaluate(x_test, te_label, batch_size=30)

2233/2233 [==============================] - 0s 206us/sample - loss: 0.3405 - sparse_categorical_accuracy: 0.9386


In [17]:
print(sklearn.metrics.classification_report(te_label,pred_yindex))

              precision    recall  f1-score   support

           0       0.96      0.98      0.97       462
           1       0.93      0.90      0.92       450
           2       0.93      0.93      0.93       450
           3       0.93      0.96      0.94       422
           4       0.95      0.92      0.93       449

    accuracy                           0.94      2233
   macro avg       0.94      0.94      0.94      2233
weighted avg       0.94      0.94      0.94      2233



In [22]:
#3
score = model.evaluate(x_test, te_label, batch_size=30)
print(sklearn.metrics.classification_report(te_label,pred_yindex))

2689/2689 [==============================] - 1s 201us/sample - loss: 0.2684 - sparse_categorical_accuracy: 0.9517
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       529
           1       0.90      0.93      0.92       544
           2       0.95      0.97      0.96       549
           3       0.99      0.95      0.97       526
           4       0.93      0.92      0.93       541

    accuracy                           0.95      2689
   macro avg       0.95      0.95      0.95      2689
weighted avg       0.95      0.95      0.95      2689



In [27]:
#4
score = model.evaluate(x_test, te_label, batch_size=30)
print(sklearn.metrics.classification_report(te_label,pred_yindex))

2087/2087 [==============================] - 0s 210us/sample - loss: 0.2914 - sparse_categorical_accuracy: 0.9420
              precision    recall  f1-score   support

           0       0.98      0.99      0.98       424
           1       0.91      0.91      0.91       409
           2       0.95      0.96      0.95       424
           3       0.92      0.98      0.95       404
           4       0.94      0.88      0.91       426

    accuracy                           0.94      2087
   macro avg       0.94      0.94      0.94      2087
weighted avg       0.94      0.94      0.94      2087



In [12]:
#5
score = model.evaluate(x_test, te_label, batch_size=30)
print(sklearn.metrics.classification_report(te_label,pred_yindex))

2159/2159 [==============================] - 0s 205us/sample - loss: 0.2663 - sparse_categorical_accuracy: 0.9444
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       413
           1       0.93      0.92      0.93       450
           2       0.95      0.97      0.96       438
           3       0.93      0.96      0.95       430
           4       0.93      0.87      0.90       428

    accuracy                           0.94      2159
   macro avg       0.94      0.94      0.94      2159
weighted avg       0.94      0.94      0.94      2159



In [35]:
#6
score = model.evaluate(x_test, te_label, batch_size=30)
print(sklearn.metrics.classification_report(te_label,pred_yindex))

2279/2279 [==============================] - 1s 255us/sample - loss: 0.2145 - sparse_categorical_accuracy: 0.9509
              precision    recall  f1-score   support

           0       0.98      0.97      0.98       447
           1       0.95      0.91      0.93       477
           2       0.94      0.98      0.96       445
           3       0.93      0.96      0.94       438
           4       0.96      0.93      0.95       472

    accuracy                           0.95      2279
   macro avg       0.95      0.95      0.95      2279
weighted avg       0.95      0.95      0.95      2279



In [39]:
#7
score = model.evaluate(x_test, te_label, batch_size=30)
print(sklearn.metrics.classification_report(te_label,pred_yindex))

2377/2377 [==============================] - 1s 216us/sample - loss: 0.1994 - sparse_categorical_accuracy: 0.9584
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       489
           1       0.94      0.96      0.95       463
           2       0.95      0.97      0.96       488
           3       0.94      0.96      0.95       463
           4       0.97      0.91      0.94       474

    accuracy                           0.96      2377
   macro avg       0.96      0.96      0.96      2377
weighted avg       0.96      0.96      0.96      2377



In [43]:
#8
score = model.evaluate(x_test, te_label, batch_size=30)
print(sklearn.metrics.classification_report(te_label,pred_yindex))

2170/2170 [==============================] - 1s 394us/sample - loss: 0.2455 - sparse_categorical_accuracy: 0.9456
              precision    recall  f1-score   support

           0       0.98      0.99      0.98       443
           1       0.91      0.96      0.93       430
           2       0.96      0.94      0.95       438
           3       0.95      0.91      0.93       432
           4       0.93      0.93      0.93       427

    accuracy                           0.95      2170
   macro avg       0.95      0.95      0.95      2170
weighted avg       0.95      0.95      0.95      2170



In [7]:
#9
score = model.evaluate(x_test, te_label, batch_size=30)
print(sklearn.metrics.classification_report(te_label,pred_yindex))

2200/2200 [==============================] - 1s 384us/sample - loss: 0.2894 - sparse_categorical_accuracy: 0.9373
              precision    recall  f1-score   support

           0       0.97      0.98      0.97       458
           1       0.89      0.90      0.89       430
           2       0.94      0.96      0.95       425
           3       0.95      0.95      0.95       459
           4       0.94      0.89      0.92       428

    accuracy                           0.94      2200
   macro avg       0.94      0.94      0.94      2200
weighted avg       0.94      0.94      0.94      2200



In [52]:
#10
score = model.evaluate(x_test, te_label, batch_size=30)
print(sklearn.metrics.classification_report(te_label,pred_yindex))

2476/2476 [==============================] - 1s 217us/sample - loss: 0.1818 - sparse_categorical_accuracy: 0.9588
              precision    recall  f1-score   support

           0       0.99      0.98      0.99       503
           1       0.92      0.94      0.93       508
           2       0.97      0.96      0.97       474
           3       0.95      0.97      0.96       481
           4       0.97      0.94      0.95       510

    accuracy                           0.96      2476
   macro avg       0.96      0.96      0.96      2476
weighted avg       0.96      0.96      0.96      2476



In [56]:
#11
score = model.evaluate(x_test, te_label, batch_size=30)
print(sklearn.metrics.classification_report(te_label,pred_yindex))

2433/2433 [==============================] - 1s 212us/sample - loss: 0.1895 - sparse_categorical_accuracy: 0.9601
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       499
           1       0.96      0.90      0.93       496
           2       0.97      0.98      0.97       482
           3       0.93      0.98      0.95       468
           4       0.95      0.94      0.95       488

    accuracy                           0.96      2433
   macro avg       0.96      0.96      0.96      2433
weighted avg       0.96      0.96      0.96      2433



In [22]:
#12
score = model.evaluate(x_test, te_label, batch_size=30)
print(sklearn.metrics.classification_report(te_label,pred_yindex))

1974/1974 [==============================] - 0s 214us/sample - loss: 0.2652 - sparse_categorical_accuracy: 0.9336
              precision    recall  f1-score   support

           0       0.96      0.99      0.97       392
           1       0.89      0.91      0.90       390
           2       0.95      0.95      0.95       417
           3       0.94      0.94      0.94       381
           4       0.92      0.88      0.90       394

    accuracy                           0.93      1974
   macro avg       0.93      0.93      0.93      1974
weighted avg       0.93      0.93      0.93      1974



In [6]:
#13
score = model.evaluate(x_test, te_label, batch_size=30)
print(sklearn.metrics.classification_report(te_label,pred_yindex))

2741/2741 [==============================] - 1s 194us/sample - loss: 0.1941 - sparse_categorical_accuracy: 0.9566
              precision    recall  f1-score   support

           0       1.00      0.99      0.99       552
           1       0.90      0.97      0.94       515
           2       0.96      0.95      0.95       588
           3       0.97      0.95      0.96       542
           4       0.95      0.93      0.94       544

    accuracy                           0.96      2741
   macro avg       0.96      0.96      0.96      2741
weighted avg       0.96      0.96      0.96      2741



In [13]:
#14
score = model.evaluate(x_test, te_label, batch_size=30)
print(sklearn.metrics.classification_report(te_label,pred_yindex))

2440/2440 [==============================] - 1s 360us/sample - loss: 0.2160 - sparse_categorical_accuracy: 0.9578
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       505
           1       0.94      0.94      0.94       484
           2       0.97      0.95      0.96       501
           3       0.94      0.96      0.95       458
           4       0.93      0.95      0.94       492

    accuracy                           0.96      2440
   macro avg       0.96      0.96      0.96      2440
weighted avg       0.96      0.96      0.96      2440



In [26]:
#15
score = model.evaluate(x_test, te_label, batch_size=30)
print(sklearn.metrics.classification_report(te_label,pred_yindex))

2107/2107 [==============================] - 0s 216us/sample - loss: 0.2061 - sparse_categorical_accuracy: 0.9473
              precision    recall  f1-score   support

           0       0.99      0.97      0.98       438
           1       0.95      0.90      0.92       422
           2       0.97      0.96      0.96       415
           3       0.89      0.95      0.92       415
           4       0.95      0.95      0.95       417

    accuracy                           0.95      2107
   macro avg       0.95      0.95      0.95      2107
weighted avg       0.95      0.95      0.95      2107

